In [1]:
import json
import re

class TextoJson:
    def __init__(self, nombreArchivo):
        self.nombreArchivo = nombreArchivo
        self.informacion = None
        # vectores
        self.data = {
            "antecedentes_personales": {},
            "antecedentes_familiares": {},
            "estadia_tumoral": {},
            "inmunohistoquimica_tumoral": {},
            "tipo": {},
        }

        # Cargar datos al inicializar el objeto
        self.cargar_datos()

    def cargar_datos(self):
        with open(self.nombreArchivo, 'r', encoding='utf-8') as file:
            self.informacion = file.readlines()

    #estructura la data a json
    def estructurar_data(self):
        return json.dumps(self.data, indent=4, ensure_ascii=False)

    def preprocesar_texto(self):
        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in self.informacion:
            linea = linea.strip().lower()

            if not linea:
                continue  # Ignorar líneas vacías

            # Identificar si la línea es una nueva clave
            if re.match(r'^[a-záéíóúñ]+.*?:', linea, re.IGNORECASE) or re.match(r'^-\s*[a-záéíóúñ]+.*?', linea, re.IGNORECASE):
                # Si ya hay una clave en curso, guardar la clave y su valor
                if clave_actual:
                    if valor_actual:  # Si tiene valores asociados
                        texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                    else:  # Si no tiene valores, solo agregar la clave
                        texto_procesado.append(clave_actual)
                
                # Actualizar clave y reiniciar el valor
                if ':' in linea:
                    # Eliminar "-" inicial y espacios en caso de que existan
                    linea = re.sub(r'^-\s*', '', linea)
                    clave_actual = linea.split(':', 1)[0].strip()
                    valor_actual = [linea.split(':', 1)[1].strip()]
                else:
                    clave_actual = linea.strip('- ').strip()
                    valor_actual = []
            elif re.match(r'^[+-]?\s*cáncer\b', linea, re.IGNORECASE):
                # Si la línea contiene "cáncer", agregarla directamente al texto procesado
                texto_procesado.append(f"diagnóstico: {linea}")
                

            elif clave_actual:
                # Agregar la línea al valor actual
                valor_actual.append(linea)
            else:
                # Este caso es cuando la clave no tiene dos puntos
                texto_procesado.append(linea)

        # Guardar la última clave-valor si quedó algo pendiente
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")

        texto_procesado = [re.sub(r'^[^a-zA-Z0-9]+', '', linea) for linea in texto_procesado]

        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)
    
    def extraer_datos_antecedentes_personales(self):
        # secciones de antecedentes_personales
        # secciones de antecedentes_personales
        antecedentes_personales_secciones = {
            "edad": None,
            "sexo": "femenino",
            "peso": None,
            "talla": None,
            "preferencia": None,
            "índice_tabáquico": None,
            "alcohol": None,
            "drogas": None,
            "comorbilidades": None,
            "antecedentes_ginecológicos": {
                "fum": None,
                "menarca": None,
                "embarazos": None,
                "partos": None,
                "trh": None,
                "estado hormonal": None,
                "metodos anticonceptivos": None
            }
        }

        #palaras clave para antecedentes_personales que buscaremos en el texto
        antecedentes_personales_keywords = [
            "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "tabaco", "tabaquismo", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos", "menarca", "embarazos", "partos", "fum", "trh", 
            "estado hormonal", "metodos anticonceptivos"
        ]

        texto_procesado = self.preprocesar_texto()
        isEdad = False

        # Agregar una regex para detectar edades en el formato "XX años"
        edad_regex = re.compile(
            r'\b(?:edad:?\s*(\d+)(?:\s*años)?)|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(\w+)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$',            re.IGNORECASE
        )
        for linea in texto_procesado.splitlines():
            # Buscar una edad en el formato específico
            match = edad_regex.match(linea)
            if match:
                if match.group(1):  # "edad: XX" o "edad: XX años"
                    antecedentes_personales_secciones["edad"] = match.group(1).strip()
                elif match.group(2):  # "XX años" o "XX años de edad"
                    antecedentes_personales_secciones["edad"] = match.group(2).strip()
                elif match.group(5):  # Formato "gacmc / ... / XX años / ..."
                    antecedentes_personales_secciones["edad"] = match.group(5).strip()
                isEdad = True
            
            for keyword in antecedentes_personales_keywords:
                if keyword in linea:

                    #si encuentra la palabra "edad"
                    if keyword == "edad":
                        if isEdad:
                            continue
                        if ":" in linea:
                            antecedentes_personales_secciones["edad"] = linea.split(":")[1].strip()
                        else:
                            antecedentes_personales_secciones["edad"] = linea.split(keyword)[1].strip()
                    
                    #si la keyword es "tabaquismo" o "tabaco" o "índice tabáquico" y si tiene o no ":" que separan_
                    if keyword == "tabaquismo" or keyword == "tabaco" or keyword == "índice tabáquico":
                        if ":" in linea:
                            antecedentes_personales_secciones["índice_tabáquico"] = linea.split(":")[1].strip()
                        else:
                            antecedentes_personales_secciones["índice_tabáquico"] = linea.split(keyword)[1].strip()

                    if keyword in ["fum", "menarca", "trh", "estado hormonal", "metodos anticonceptivos", "embarazos", "partos"]:
                        # Verificar las keywords de antecedentes ginecológicos en la misma línea
                        pattern = re.compile(r'(fum|menarca|trh|estado hormonal|metodos anticonceptivos|embarazos|partos)', re.IGNORECASE)
                        matches = pattern.findall(linea)
                        if len(matches) > 1:
                           # Dividir la línea en partes basadas en varios separadores posibles
                            partes = re.split(r'[\/,\.]', linea)
                            for parte in partes:
                                # Asegurarte de que cada elemento sea una cadena válida
                                if isinstance(parte, str):  # Por seguridad, verifica si el elemento es una cadena
                                    parte = parte.strip()  # Elimina espacios al inicio y al final
                                    
                                for keyword in matches:
                                    if keyword in parte:
                                        if ":" in parte:
                                            antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(":")[1].strip()
                                        else:
                                            # Verificar si el valor precede a la clave
                                            if parte.strip().startswith(keyword):
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[1].strip()
                                            else:
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[0].strip()
                        else:
                            if ":" in linea:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword] = linea.split(":")[1].strip()
                            else:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword] = linea.split(keyword)[1].strip()
                    
                    ##tratamos las comoorbilidades dado que es una linea con varias comorbilidades separadas por "/"
                    if keyword == "comorbilidades":
                        comorbilidades = linea
                        # Dividir la cadena en comorbilidades individuales
                        comorbilidades_list = re.split(r'\d+\.\s', comorbilidades)

                        # Eliminar posibles cadenas vacías resultantes de la división
                        comorbilidades_list = [comorbilidad for comorbilidad in comorbilidades_list if comorbilidad]

                        comorbilidades_dict = {}

                        for comorbilidad in comorbilidades_list:
                            comorbilidad = comorbilidad.strip()
                            # Extraer el año
                            match = re.search(r'\((\d{4})\)', comorbilidad)
                            if match:
                                year = match.group(1)
                                # Separar nombre y descripción
                                parts = comorbilidad.split(f"({year})")
                                nombre = parts[0].strip()
                                descripcion = parts[1].strip() if len(parts) > 1 else ""

                                # Si la descripción es solo un punto, lo agregamos al nombre
                                if descripcion == ".":
                                    descripcion = None  # Dejamos la descripción vacía
                                    nombre = nombre.rstrip('.')  # Quitamos el punto del nombre

                                if year not in comorbilidades_dict:
                                    comorbilidades_dict[year] = []

                                comorbilidades_dict[year].append({
                                    "nombre": nombre,
                                    "descripcion": descripcion.lstrip(': ') if descripcion else None
                                })

                        # Imprimir el resultado
                        antecedentes_personales_secciones["comorbilidades"] = comorbilidades_dict

                    
   
        #para debug pasamos antecedentes_personales_secciones a json
        print(json.dumps(antecedentes_personales_secciones, indent=4, ensure_ascii=False))
               

        #print(self.estructurar_data())
    

procesador = TextoJson('/home/josuevj/Documents/uni/servicio/sources/OCR_erik/ExtraccionTexto/TextoExtraido2.txt')
procesador.extraer_datos_antecedentes_personales()

mlaa / 314262 / 86 años / dr. soto
comorbilidades: hipertensión arterial sistémica (1995).
fum: 47 años
menarca a los 14 años. 2 embarazos, 2 partos.
métodos anticonceptivos desconoce.
cirugías: apendicectomía a los 20 años.
tabaquismo negado
exposición al humo de leña por 40 años por 2
horas.
ahf oncológicos: negados
originaria y residente: puebla
seguridad social: 1 (imss)
diagnóstico: cáncer de mama bilateral. mi cd! pt1c, pn1a, mo / md cli pt2, cno mo rh+ iher2-
ocupación: hogar
{
    "edad": "86 años",
    "sexo": "femenino",
    "peso": null,
    "talla": null,
    "preferencia": null,
    "índice_tabáquico": "negado",
    "alcohol": null,
    "drogas": null,
    "comorbilidades": {
        "1995": [
            {
                "nombre": "comorbilidades: hipertensión arterial sistémica",
                "descripcion": null
            }
        ]
    },
    "antecedentes_ginecológicos": {
        "fum": "47 años",
        "menarca": "a los 14 años",
        "embarazos": "2",
  